In [ ]:
from IPython.display import display, HTML
import xarray
import rioxarray
from geocoreg import visualization as vz, xr_registration

In [ ]:
# Load sample data
da = rioxarray.open_rasterio("naip_example.tif")
da = da.assign_coords(band=['red', 'green', 'blue'])
# Simulate a time series with some shifts
shifted_series = xarray.concat(
    [
        da.assign_coords(time=0),
        da.shift(x=10, y=10).assign_coords(time=1),
        da.shift(x=-5, y=8).assign_coords(time=2),
    ],
    dim="time",
)
# Export a gif with the original series
vz.export_animation(shifted_series, 'time', file_name='original_series.gif')

In [ ]:
# Define the reference image as the first image of the time series
dst_img = shifted_series.isel(time=0)
# Coregistrate the time series with the reference image
ts_registrated = xr_registration.coregistrate(shifted_series, dst_img, registrator='pcc', registration_band='red')
# Export a gif with the registrated series
vz.export_animation(ts_registrated, 'time', file_name='registrated_series.gif')

In [7]:
# Once the gifs are created, we can display them in a HTML format
# Note that after displaying the first time, the gifs look to be cached
# and the changes are not displayed, unless you rename the images.
original = 'original_series.gif'
registrated = 'registrated_series.gif'

html_code = f"""
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <h3>Before registration</h3>
        <img src="{original}" width="500">
    </div>
    <div style="text-align: center;">
        <h3>After registration</h3>
        <img src="{registrated}" width="500">
    </div>
</div>
"""

# Display the HTML
display(HTML(html_code))